# 深度生成模型总结--原始GAN和VAE

前段时间研究了一段`Zero Shot Learning`，研究了一些生成模型，总结一下备忘。

关于生成模型有一个很好的[github仓库](https://github.com/wiseodd/generative-models)，而且作者的[博客](https://wiseodd.github.io/)写的也非常精彩。

所谓生成模型，即，给定一些数据，用一个向量$x$表示，每一个datapoint对应一张图片或者一句话，生成模型的目标是学会$P(x)$，有了$P(x)$就可以从中sample，从而**生成**一些数据。

本文主要总结下**GAN**和**VAE**两种生成模型。

## GAN

GAN的原始paper是大神Goodfellow的[《Generative Adversarial Nets》](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf)

先简介下模型的**思想**：

GAN包含：**一个生成模型$G$**和**一个判别模型$D$**，$G$的输入是一个随机扰动$z$，输出是生成的数据$\hat x$；$D$的输入是原始数据$x$或者$G$生成的数据$\hat x$，输出是**输入来自训练数据的概率**。

如果将$G$看作假币生产者，$D$看作警察，那么作假者的任务是尽量生产可以以假换真的假币，警察的任务是尽可能区分假币和真币，二者将不断地竞争直到生产者生产的假币完全以假乱真。

![](https://github.com/applenob/deep_learning_note/raw/master/res/gan_1.png)

**目标函数**：
$$\underset{G}{min}\underset{D}{max}\;V(D, G) = E_{x∼p_{data}(x)}[log D(x)] + E_{z∼p_z(z)}[log(1 − D(G(z)))]$$

上面的目标函数修改自**交叉熵**损失函数。

很多初次接触GAN的同学都会对GAN的训练感到迷惑，看下面的**训练算法**：

- 每轮训练迭代：
    - 前k步，训练分类器：
        - 从先验噪声：$p_g(z)$中采样m个**噪声**作为minibatch：$\{z^{(1)}, ..., z^{(m)}\}$。
        - 从数据生成分布中$p_{data}(x)$采样m个**真实数据**作为minibatch：$\{x^{(1)}, ..., x^{(m)}\}$。
        - 使用随机梯度**上升**更新**判别器的参数**：
            - $\bigtriangledown _{\theta_d}\frac{1}{m}\sum_{i=1}^m[logD(x^{(i)})+log(D(G(z^{(i)})))]$
    - 从先验噪声：$p_g(z)$中采样m个**噪声**作为minibatch：$\{z^{(1)}, ..., z^{(m)}\}$。
    - 使用随机梯度**下降**更新**生成器的参数**：
        - $\bigtriangledown _{\theta_g}\frac{1}{m}\sum_{i=1}^mlog(D(G(z^{(i)})))$
        
虽然是同一个损失函数按照相反的方向训练，但训练判别器和生成器时针对的**参数**不同（上面算法中$\theta_d$和$\theta_g$），因此并不会出现南辕北辙的情况。

**优缺点**:
- 缺点：没有明确的$p_g(x)$的表示；$D$的训练需要和$G$同步。
- 优点：采样时无需Markov chain；训练时无需inference，模型设计的范围更广。
![](https://github.com/applenob/deep_learning_note/raw/master/res/gan_cmp.png)

下面代码来自https://github.com/wiseodd/generative-models ，使用python3.5。

In [1]:
%matplotlib inline
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from tensorflow.examples.tutorials.mnist import input_data


mnist = input_data.read_data_sets('data', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
c = 0
lr = 1e-3


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)


""" ==================== GENERATOR ======================== """

Wzh = xavier_init(size=[Z_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)


def G(z):
    """一个简单的双层神经网络作为生成器"""
    h = nn.relu(z @ Wzh + bzh.repeat(z.size(0), 1))  # @是python3中新加入的矩阵乘法符号
    X = nn.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X


""" ==================== DISCRIMINATOR ======================== """

Wxh = xavier_init(size=[X_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Why = xavier_init(size=[h_dim, 1])
bhy = Variable(torch.zeros(1), requires_grad=True)


def D(X):
    """一个简单的双层神经网络作为生成器"""
    h = nn.relu(X @ Wxh + bxh.repeat(X.size(0), 1))
    y = nn.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    return y


G_params = [Wzh, bzh, Whx, bhx]
D_params = [Wxh, bxh, Why, bhy]
params = G_params + D_params


""" ===================== TRAINING ======================== """


def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())

# 优化器指定参数
G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))


for it in range(10000):
    # Sample data
    z = Variable(torch.randn(mb_size, Z_dim))
    X, _ = mnist.train.next_batch(mb_size)
    X = Variable(torch.from_numpy(X))

    # Dicriminator forward-loss-backward-update
    G_sample = G(z)
    D_real = D(X)
    D_fake = D(G_sample)

    D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
    D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
    D_loss = D_loss_real + D_loss_fake

    D_loss.backward()
    D_solver.step()

    # Housekeeping - reset gradient
    reset_grad()

    # Generator forward-loss-backward-update
    z = Variable(torch.randn(mb_size, Z_dim))
    G_sample = G(z)
    D_fake = D(G_sample)

    G_loss = nn.binary_cross_entropy(D_fake, ones_label)

    G_loss.backward()
    G_solver.step()

    # Housekeeping - reset gradient
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))

        # 取前16张图保存
        samples = G(z).data.numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
        c += 1
        plt.close(fig)


Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


/home/cer/anaconda2/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Iter-0; D_loss: [ 1.50280118]; G_loss: [ 1.64008629]
Iter-1000; D_loss: [ 0.0023118]; G_loss: [ 8.38419437]
Iter-2000; D_loss: [ 0.00522094]; G_loss: [ 7.93807125]
Iter-3000; D_loss: [ 0.03023708]; G_loss: [ 4.29144192]
Iter-4000; D_loss: [ 0.02870584]; G_loss: [ 6.23502254]
Iter-5000; D_loss: [ 0.15886861]; G_loss: [ 4.66118479]
Iter-6000; D_loss: [ 0.26347995]; G_loss: [ 4.20204782]
Iter-7000; D_loss: [ 0.51319796]; G_loss: [ 2.98293495]
Iter-8000; D_loss: [ 0.70317507]; G_loss: [ 2.42791796]
Iter-9000; D_loss: [ 0.93306208]; G_loss: [ 2.59884763]


## VAE

关于VAE可以直接看这篇[《Tutorial on Variational Autoencoders》](https://arxiv.org/pdf/1606.05908.pdf)。

